<a href="https://colab.research.google.com/github/principioincertidumbre/ComputerVision/blob/main/IntroCV_Tarea_2_Sof%C3%ADa_Vits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <h1>Introducción a la Visión Computacional</h1>
    <h1>Tarea 2</h1>
</center>

### Magíster en Data Science
### U. del Desarrollo

-----------------------
<div align="right">
Fecha de Entrega: Lunes 5, Mayo 20245.
</div>

## Instrucciones

1. Diseñar un flujo de procesamiento a nivel abstracto: adquisición de imágenes, pre-procesamiento, procesamiento de imágenes, cálculo de la medición, almacenamiento.

2. Con la base de datos seleccionada, implementar un sistema de reconocimiento y/o procesamiento en el cual se realice una medida o inferencia sobre imagen. Puede ser la detección de un objeto, medición de distancias entre píxeles de alguna característica, medición del tamaño de algún objeto (en cantidad de píxeles) clasificación de la imagen, entre otros.

3. Para procesar el conjunto completo de imágenes, medir los tiempos de ejecución del proceso completo. Realizar la medición para el 50% y el 100% de la imágenes varias veces, de manera de obtener algunas figuras estadísticas: tiempo máximo, tiempo mínimo, tiempo medio, desviación estándar del tiempo.

4. Hacer resumen de los resultados del procesamiento. Para ello debe decidir una métrica para indicar si el procesamiento es satisfactorio. Para esto último algunas veces es necesario hacer la evaluación en forma manual, por lo tanto, si fuera este el caso, seleccionar un sub-conjunto de las imágenes (por ejemplo, unas 50) y comparar el desempeño del algoritmo en estas imágenes.

## Formato de Entrega:
1. Escribir un informe o entregar el notebook con comentarios y documentación, incluyendo el análisis (comentando) los resultados. Recordar las recomendaciones de la Tarea 1, de **no mostrar todas las imágenes procesadas en el notebook** sino que solamente un número reducido de ejemplos. Los comentarios son muy importantes.

2. De percatarse que si encontrarn que el set de imágenes propuesto en la Tarea 1 no es el adecuado, se puede realizar un cambio en el dataset. Justificar el cambio. Este tipo de decisiones puede ocurrir en un proyecto. Muchas veces es necesario probar una variedad de datasets.

3. No está permitido el uso de Herramientas de Inteligencia Artificial para generar el informe. El uso de Inteligencia Artificial Generativa solamente se remite a la sugerencia de código parcial, como copilot.

# Introducción

# Metodología

# Desarrollo

# Conclusiones